## In this notebook, I will be analyzing the neighborhoods of Toronto and determine the area that is most suitable such that it has many amenities for a family migrating from China such as schools, restaurants etc.

## In simple terms, I can describe my project as the determination of a neighborhood that has major number of Chinese amenities.

In [12]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 


import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [13]:
!pip install lxml

#### First, I will be cleaning the data that we have obtained from Wikipedia with regards to the neighborhoods in Toronto

In [14]:
import pandas as pd
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
toronto = pd.read_html(url)
df_toronto = pd.DataFrame(toronto[0])
print(df_toronto.shape)
df_toronto.head()

(288, 3)


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [15]:
df_toronto = df_toronto.set_index('Borough')
df_toronto = df_toronto.drop('Not assigned', axis = 0)
df_toronto = df_toronto.reset_index(drop=False)
print(df_toronto.shape)
df_toronto.head()

(211, 3)


,Borough,Postcode,Neighbourhood
0,North York,M3A,Parkwoods
1,North York,M4A,Victoria Village
2,Downtown Toronto,M5A,Harbourfront
3,Downtown Toronto,M5A,Regent Park
4,North York,M6A,Lawrence Heights


In [16]:
df_toronto['Neighbourhood'] = df_toronto['Neighbourhood'].replace('Not assigned', df_toronto['Borough'])
print(df_toronto.shape)
df_toronto.head()

(211, 3)


,Borough,Postcode,Neighbourhood
0,North York,M3A,Parkwoods
1,North York,M4A,Victoria Village
2,Downtown Toronto,M5A,Harbourfront
3,Downtown Toronto,M5A,Regent Park
4,North York,M6A,Lawrence Heights


In [17]:
# Combine rows with same postal code
df_neighbourhood = df_toronto.groupby(['Postcode','Borough'], sort = False).agg(lambda x: ', '.join(x)).reset_index()
print(df_neighbourhood.shape)
df_neighbourhood.head()

(103, 3)


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


#### Let us process the data using co-ordinates we have obtained from a csv file

In [18]:
toronto_geocsv = 'https://cocl.us/Geospatial_data'
!wget -q -O 'toronto_m.geospatial_data.csv' toronto_geocsv
geocsv_data = pd.read_csv(toronto_geocsv)


In [19]:
df_latlon = pd.DataFrame(geocsv_data)
print(df_latlon.shape)
df_latlon.head()

(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
df_latlon.rename(columns={'Postal Code': 'Postcode'}, inplace=True)
df_latlon.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [21]:
df_merged = pd.merge(df_neighbourhood, df_latlon, on='Postcode')
print(df_merged.shape)
df_merged.head()

(103, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [22]:
print('The city has {} boroughs and {} postal code areas' .format(len(df_merged['Borough'].unique()), df_merged.shape[0]))

The city has 11 boroughs and 103 postal code areas


#### Having completed the cleaning of the data, we can now proceed to creating a visual map of the data to our requirements

In [23]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [24]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Borough'], df_merged['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Using foursquareAPI for exploration

In [25]:
# The code was removed by Watson Studio for sharing.

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [27]:
toronto_venues = getNearbyVenues(names=df_merged['Neighbourhood'],
                                   latitudes=df_merged['Latitude'],
                                   longitudes=df_merged['Longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront, Regent Park
Lawrence Heights, Lawrence Manor
Queen's Park
Islington Avenue
Rouge, Malvern
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Highland Creek, Rouge Hill, Port Union
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Thorncliffe Park
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
East Birchmount Park, Ionview, Kennedy Park
Bayview Village
CFB Toronto, Downsview East
The D

In [28]:
print(toronto_venues.shape)
toronto_venues.head()

(2262, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [29]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",9,9,9,9,9,9
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Downsview North, Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",22,22,22,22,22,22
Berczy Park,57,57,57,57,57,57


#### Let's check how many different categories and how many Chinese restaurants were found

In [30]:
print('There are {} different venue categories' .format(len(toronto_venues['Venue Category'].unique())))


There are 272 different venue categories


In [31]:
toronto_venues['Venue Category'].value_counts()

Coffee Shop                        193
Café                               102
Restaurant                          59
Park                                56
Pizza Place                         54
Italian Restaurant                  53
Bakery                              45
Hotel                               44
Sandwich Place                      43
Bar                                 42
Clothing Store                      36
Fast Food Restaurant                33
Japanese Restaurant                 31
Gym                                 31
American Restaurant                 29
Burger Joint                        27
Pub                                 27
Breakfast Spot                      26
Sushi Restaurant                    25
Grocery Store                       24
Gastropub                           24
Seafood Restaurant                  24
Thai Restaurant                     24
Steakhouse                          23
Chinese Restaurant                  23
Ice Cream Shop           

In [32]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to front
cols = list(toronto_onehot)
cols.insert(0, cols.pop(cols.index('Neighborhood')))
toronto_onehot = toronto_onehot.ix[:, cols]
toronto_onehot.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [33]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').sum().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0,0,0,0,0,0,0,0,4,0,0,1,1,0,3,0,0,0,0,0,0,1,0,4,0,0,0,0,0,0,0,

### For my project, I have considered the suitable area with regards to most Chinese restaurants

In [34]:
df_ChineseRest = toronto_grouped[['Neighborhood', 'Chinese Restaurant']].sort_values(by='Chinese Restaurant', ascending=False)
df_ChineseRest.head()

,Neighborhood,Chinese Restaurant
20,"Chinatown, Grange Park, Kensington Market",5
19,Central Bay Street,2
60,L'Amoreaux West,2
83,Studio District,1
68,North Toronto West,1


In [35]:
df_merged_CGKM = df_merged[df_merged['Neighbourhood']=='Chinatown, Grange Park, Kensington Market']
df_merged_CGKM

,Postcode,Borough,Neighbourhood,Latitude,Longitude
84,M5T,Downtown Toronto,"Chinatown, Grange Park, Kensington Market",43.653206,-79.400049


In [36]:
# Define the latitude and longitude value of the neighbourhood with the most Chinese Restaurants
# to display it later on the map
CW_lat = 43.66586
CW_lng = -79.38316

In [37]:
search_query = 'Chinese'
#= '4d4b7104d754a06370d81259'
radius = 50000
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    latitude, 
    longitude, 
    VERSION, 
    search_query, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=2IPLGG2NJLA3R0L23C3DPZKQKOMJC0IDNTAUEHBOKHBLQGWW&client_secret=WJF4LSXMJEQLDL2VT5BCQQXF052HGA3CJ2KRVWM5MVVV1RAG&ll=43.653963,-79.387207&v=20180605&query=Chinese&radius=50000&limit=100'

In [38]:
results = requests.get(url).json()

In [39]:
ChineseRest_json = results['response']['venues']

# tranform data into a dataframe
ChineseRest = json_normalize(ChineseRest_json)
ChineseRest.head()
ChineseRest.shape

(50, 19)

In [40]:
# keep only columns that include venue name, and anything that is associated with location
ChineseRest_filtered = ['name', 'categories'] + [col for col in ChineseRest.columns if col.startswith('location.')] + ['id']
ChineseRest_filtered = ChineseRest.loc[:, ChineseRest_filtered]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
ChineseRest_filtered['categories'] = ChineseRest_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
ChineseRest_filtered.columns = [column.split('.')[-1] for column in ChineseRest_filtered.columns]

ChineseRest_filtered.head(50)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,ILSC-Toronto Chinese Catholic Centre Campus,Language School,202 St Patrick St,CA,Toronto,Canada,NaN,335,"[202 St Patrick St, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65542737655215...",43.655427,-79.390848,NaN,NaN,ON,53973080498e97c3af5edb92
1,Rosewood Chinese Cuisine,Chinese Restaurant,463 Dundas St. W.,CA,Toronto,Canada,at Larch St.,770,"[463 Dundas St. W. (at Larch St.), Toronto ON ...","[{'label': 'display', 'lat': 43.65317123090316...",43.653171,-79.396710,NaN,M5T 1G8,ON,4fd3ea3ee4b0191b9c2b1aff
2,Sky Dragon Chinese Restaurant 龍翔酒樓,Dim Sum Restaurant,280 Spadina Ave.,CA,Toronto,Canada,at Dundas St. W.,892,"[280 Spadina Ave. (at Dundas St. W.), Toronto ...","[{'label': 'display', 'lat': 43.65278331265585...",43.652783,-79.398174,NaN,NaN,ON,4b072e9df964a52009f922e3
3,Chinese Visa Application Service Center,Government Building,"393 University Ave, Suite 1501",CA,Toronto,Canada,in University Centre,14,"[393 University Ave, Suite 1501 (in University...","[{'label': 'display', 'lat': 43.65402839343005...",43.654028,-79.387365,NaN,NaN,ON,4f04779a02d5cce0cfc06151
4,Chinese Traditional Buns,Chinese Restaurant,536 Dundas St. W,CA,Toronto,Canada,at Spadina Ave,960,"[536 Dundas St. W (at Spadina Ave), Toronto ON...","[{'label': 'display', 'lat': 43.65271411616852...",43.652714,-79.399006,Kensington Market,M5T 1H3,ON,4b622bf1f964a5200b3a2ae3
5,North-East Chinese Restaurant 華北美食,Chinese Restaurant,476 Dundas St.,CA,Toronto,Canada,Huron,767,"[476 Dundas St. (Huron), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.6531846832903,...",43.653185,-79.396677,NaN,NaN,ON,4d06c1ccc2e53704428cc567
6,Hong Shing Chinese Restaurant,Chinese Restaurant,195 Dundas St W,CA,Toronto,Canada,at University Ave,107,"[195 Dundas St W (at University Ave), Toronto ...","[{'label': 'display', 'lat': 43.65492521335936...",43.654925,-79.387089,NaN,M5G 1C7,ON,4b2027b5f964a520f82d24e3
7,Wok & Roast Chinese BBQ,Chinese Restaurant,349 Broadview Ave,CA,Toronto,Canada,Gerrard St E,3071,"[349 Broadview Ave (Gerrard St E), Toronto ON ...","[{'label': 'display', 'lat': 43.665067, 'lng':...",43.665067,-79.352298,NaN,M4M 2H1,ON,58ab6f3eea29b818ab66cb3e
8,Chinese Bakery,Bakery,433 Dundas St. W,CA,Toronto,Canada,at Huron St.,678,"[433 Dundas St. W (at Huron St.), Toronto ON M...","[{'label': 'display', 'lat': 43.65344583913006...",43.653446,-79.395601,NaN,M5T 1G6,ON,4af89f6ff964a520c30e22e3
9,Toronto Chinese Academy,University,133 Richmond Street West,CA,Toronto,Canada,York Street,465,"[133 Richmond Street West (York Street), Toron...","[{'label': 'display', 'lat': 43.65014006138457...",43.650140,-79.384857,Financial District,M5H 2L3,ON,5202d246498e9649c88150b7


In [41]:
ChineseRest_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [CW_lat, CW_lng],
    radius=23,
    color='red',
    popup='Centre Point',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(ChineseRest_map)

# add the Chinese restaurants as blue circle markers
for lat, lng, label in zip(ChineseRest_filtered.lat, ChineseRest_filtered.lng, ChineseRest_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(ChineseRest_map)

# display map
ChineseRest_map

## Result:

#### As per my findings above, I have determined the neighborhood for  people with Chinese ethnicities to live in due to the presence of various amenities such as restaurants, educational institutes etc.

## Conclusion:

#### I used Foursquare API to identify all the Chinese restaurants in Toronto and grouped them by the neighbourhood in which they're located. By doing that I identified "Chinatown, Grange Park, Kensington Market" as the most suitable neighbourhood in regards to my original purpose. By displaying the results on a map I can see "Chinatown, Grange Park, Kensington Market" is an area with some Chinese Restaurants.

### NOTE: For a business analysis on the place to open a Chinese restaurant as well, we can use the data obtained above. The best location for such a venture would be a locality that has few or no Chinese restaurants. From the table above, we can determine that  a neighborhood such as North Toronto West could be ideal for such a venture as there will be less competition when compared to a neighborhood such as Chinatown, Grange Park, Kensington Market.